# question 2

In [1]:
from os import path
import os
import pandas as pd
import numpy as np
import gurobipy as gb
from gurobipy import GRB
from gurobipy import quicksum
import math

In [2]:
os.getcwd()

'/home/eason/109-2-OperationReseach/Case2'

In [3]:
work_dir = os.getcwd()
data_path = path.join(work_dir, 'data')
data_file = path.join(data_path, 's3.xlsx')

In [4]:
Demand_df = pd.read_excel(data_file, sheet_name='Demand', index_col='Product')
Init_df = pd.read_excel(data_file, sheet_name='Initial inventory', index_col='Product')
ShippingCost_df = pd.read_excel(data_file, sheet_name='Shipping cost', index_col='Product')
InTransit_df = pd.read_excel(data_file, sheet_name='In-transit', index_col='Product')
CBM_df = pd.read_excel(data_file, sheet_name='Size', index_col='Product')
PriceAndCost_df = pd.read_excel(data_file, sheet_name='Price and cost', index_col='Product')
Shortage_df = pd.read_excel(data_file, sheet_name='Shortage', index_col='Product')
VendorProduct_df = pd.read_excel(data_file, sheet_name='Vendor-Product', index_col='Product') - 1
VendorCost_df = pd.read_excel(data_file, sheet_name='Vendor cost', index_col='Vendor')
Bound_df = pd.read_excel(data_file, sheet_name='Bounds', index_col='Product')
Conflict_df = pd.read_excel(data_file, sheet_name='Conflict', index_col='Conflict') - 1

N, M = Demand_df.shape
K = 3 #(express delivery, air freight, Ocean freight)
V = VendorCost_df.shape[0]
ContainerCap = 30 #Case 1 data
ContainerCost = 2750 #Case 1 data

Demand_ij = Demand_df.to_numpy()
Init_i = Init_df.to_numpy().squeeze()
BuyCost_i = PriceAndCost_df['Purchasing cost'].to_numpy()
HoldCost_i = PriceAndCost_df['Holding cost'].to_numpy()
Transit_ij = InTransit_df.to_numpy()
ShipFixedCost_k = np.array([100, 80, 50]) #Case 1 data
ShipVarCost_ik = np.concatenate(( ShippingCost_df.to_numpy(), np.zeros((N,1)) ), axis=1)
CBM_i = CBM_df.to_numpy().squeeze()
LostSaleCost_i = Shortage_df['Lost sales'].to_numpy()
BackOrderCost_i = Shortage_df['Backorder'].to_numpy()
BackOrderProb_i = Shortage_df['Backorder percentage'].to_numpy()
VendorFixedCost_v = VendorCost_df.to_numpy().squeeze()
MinOrder_i = Bound_df.to_numpy().squeeze()
ConflictPair_alpha = Conflict_df.to_numpy()
ProductVendor_i = VendorProduct_df.to_numpy().squeeze()
M_big = np.sum(Demand_ij) + sum(Init_i)

In [5]:
#model
instance_name = 's3'
model = gb.Model(name=instance_name)

Academic license - for non-commercial use only - expires 2021-06-26
Using license file /home/eason/gurobi.lic


In [6]:
#decision variable
x = model.addVars(N, M, K, vtype=GRB.INTEGER,name='x')
Abin = model.addVars(M, K, vtype=GRB.BINARY, name='Abin')
ContainerCnt = model.addVars(M, vtype=GRB.INTEGER, name='ContainerCnt')
StockLevel = model.addVars(N, M, vtype=GRB.CONTINUOUS, name='StockLevel')
Shortage = model.addVars(N, M, vtype=GRB.CONTINUOUS, name='Shortage')
Bbin = model.addVars(N, M, vtype=GRB.BINARY, name='Bbin')
Cbin = model.addVars(N, M, vtype=GRB.BINARY, name='Cbin')
Dbin = model.addVars(V, M, vtype=GRB.BINARY, name='Dbin')

In [7]:
#Expr
VolumeInOceanExpr_j = [
gb.LinExpr(
    quicksum(x[i,j,2] * CBM_i[i] for i in range(N))
)    
for j in range(M)]

BackOrderCntExpr_ij = [
    [
        gb.LinExpr( Shortage[i,j] * BackOrderProb_i[i] )
    for j in range(M)]
for i in range(N)]

LostSaleCntExpr_ij = [
    [
        gb.LinExpr( Shortage[i,j] * (1 - BackOrderProb_i[i]) )
    for j in range(M)]
for i in range(N)]

In [8]:
#obj function
TotalPurchaseCost = gb.LinExpr(
    quicksum(
        quicksum(
            quicksum(
                x[i,j,k] * BuyCost_i[i]
            for i in range(N))
        for j in range(M))
    for k in range(K))
)

TotalShipFixedCost = gb.LinExpr(
    quicksum(
        quicksum(
            Abin[j,k] * ShipFixedCost_k[k]
        for k in range(K))
    for j in range(M))
)

TotalShipVarCost = gb.LinExpr(
    quicksum(
        quicksum(
            quicksum(
                x[i,j,k] * ShipVarCost_ik[i,k]
            for k in range(K))
        for j in range(M))
    for i in range(N))
)

TotalHoldingCost = gb.LinExpr(
    quicksum(
        quicksum(
            StockLevel[i,j] * HoldCost_i[i]
        for i in range(N))
    for j in range(M))
)

TotalContainerCost = gb.LinExpr(
    quicksum(ContainerCnt[j] for j in range(M)) * ContainerCost
)

TotalBackOrderCost = gb.LinExpr(
    quicksum(
        quicksum(
            BackOrderCntExpr_ij[i][j] * BackOrderCost_i[i]
        for i in range(N))
    for j in range(M))
)

TotalLostSaleCost = gb.LinExpr(
    quicksum(
        quicksum(
            LostSaleCntExpr_ij[i][j] * LostSaleCost_i[i]
        for i in range(N))
    for j in range(M))
)

TotalVendorFixedCost = gb.LinExpr(
    quicksum(
        quicksum(
            Dbin[v,j] * VendorFixedCost_v[v]
        for v in range(V))
    for j in range(M))
)

In [9]:
#set obj function
model.setObjective(
    TotalPurchaseCost +
    TotalShipFixedCost + 
    TotalShipVarCost + 
    TotalHoldingCost + 
    TotalContainerCost + 
    TotalBackOrderCost + 
    TotalLostSaleCost +
    TotalVendorFixedCost
)

In [10]:
#Contrain

#let Abin to behave correctly
_ = model.addConstrs(
    quicksum(x[i,j,k] for i in range(N)) / M_big <= Abin[j,k]
for j in range(M)
for k in range(K))

#let ContainerCnt to behave correctly
_ = model.addConstrs(
    VolumeInOceanExpr_j[j] / ContainerCap <= ContainerCnt[j]
for j in range(M))

#let Stocklevle & Shortage behave correctly
_ = model.addConstrs(StockLevel[i,0] - Shortage[i,0] == Init_i[i] - Demand_ij[i][0] for i in range(N))
_ = model.addConstrs(StockLevel[i,1] - Shortage[i,1] == StockLevel[i,0] + x[i,0,0] + Transit_ij[i][1] - Demand_ij[i][1] - Shortage[i,0] * BackOrderProb_i[i] for i in range(N))
_ = model.addConstrs(StockLevel[i,2] - Shortage[i,2] == StockLevel[i,1] + x[i,1,0] + x[i,0,1] + Transit_ij[i][2] - Demand_ij[i][2] - Shortage[i,1] * BackOrderProb_i[i] for i in range(N))
_ = model.addConstrs(StockLevel[i,j] - Shortage[i,j] == StockLevel[i,j-1] + x[i,j-1,0] + x[i,j-2,1] + x[i,j-3,2] - Demand_ij[i][j] - Shortage[i,j-1] * BackOrderProb_i[i] for i in range(N) for j in range(3,M))

_ = model.addConstrs(StockLevel[i,j] <= M_big * (1-Bbin[i,j]) for i in range(N) for j in range(M))
_ = model.addConstrs(Shortage[i,j] <= M_big * Bbin[i,j] for i in range(N) for j in range(M))

#let Cbin behave correctly
_ = model.addConstrs(
    quicksum(x[i,j,k] for k in range(K)) / M_big <= Cbin[i,j]
for i in range(N)
for j in range(M))

#let Dbin behave correctly
_ = model.addConstrs(
    quicksum(
        quicksum(x[i,j,k] for k in range(K))
    for i in range(N) if ProductVendor_i[i] == v) / M_big <= Dbin[v,j]
for v in range(V)
for j in range(M))

#Minumin order Bound
_ = model.addConstrs(
    Cbin[i,j] * MinOrder_i[i] <= quicksum(x[i,j,k] for k in range(K))
for i in range(N)
for j in range(M))

#conflict
_ = model.addConstrs(
    Cbin[a,j] + Cbin[b,j] <= 1
for j in range(M)
for a, b in ConflictPair_alpha)

In [11]:
model.optimize()

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (linux64)
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads
Optimize a model with 65988 rows, 91338 columns and 327308 nonzeros
Model fingerprint: 0x7abab59e
Variable types: 26000 continuous, 65338 integer (26312 binary)
Coefficient statistics:
  Matrix range     [5e-07, 2e+06]
  Objective range  [2e+00, 4e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+06]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve removed 15606 rows and 16517 columns
Presolve time: 1.23s
Presolved: 50382 rows, 74821 columns, 274918 nonzeros
Variable types: 20788 continuous, 54033 integer (15804 binary)
Found heuristic solution: objective 1.940131e+09

Deterministic concurrent LP optimizer: primal and dual simplex
Showing first log only...

Extra simplex iterations after uncrush: 548
Concurrent spin time: 0.07s

Solved with primal simplex

Root relaxa

In [12]:
print('Express delivery')
df = pd.DataFrame([[int(x[i,j,0].x) for j in range(M)] for i in range(N)], columns=range(M))
display(df)
print('Air frieght')
df = pd.DataFrame([[int(x[i,j,1].x) for j in range(M)] for i in range(N)], columns=range(M))
display(df)
print('Ocean frieght')
df = pd.DataFrame([[int(x[i,j,2].x) for j in range(M)] for i in range(N)], columns=range(M))
display(df)

Express delivery


,0,1,2,3,4,5,6,7,8,9,...,16,17,18,19,20,21,22,23,24,25
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
496,249,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
497,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
498,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Air frieght


,0,1,2,3,4,5,6,7,8,9,...,16,17,18,19,20,21,22,23,24,25
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
496,159,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
497,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,32,0,0,0,0
498,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Ocean frieght


,0,1,2,3,4,5,6,7,8,9,...,16,17,18,19,20,21,22,23,24,25
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,97,367,0,92,175,0,285,0,278,...,0,0,0,0,0,0,0,0,0,0
4,0,67,49,51,46,69,132,277,63,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,174,300,259,0,28,261,204,359,214,0,...,64,200,201,392,201,6,48,0,0,0
496,42,82,35,22,279,188,0,162,56,91,...,226,251,21,278,0,0,0,0,0,0
497,0,0,0,0,0,0,0,0,0,0,...,141,128,287,385,254,38,0,0,0,0
498,0,0,0,0,166,189,105,164,298,55,...,252,72,139,269,257,0,244,0,0,0


In [13]:
print('StockLevel')
df = pd.DataFrame([[StockLevel[i,j].x for j in range(M)] for i in range(N)], columns=range(M))
display(df)
print('Shortage')
df = pd.DataFrame([[Shortage[i,j].x for j in range(M)] for i in range(N)], columns=range(M))
display(df)

StockLevel


,0,1,2,3,4,5,6,7,8,9,...,16,17,18,19,20,21,22,23,24,25
0,662.0,607.0,435.0,241.0,147.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,410.0,357.0,289.0,104.0,91.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,346.0,167.0,166.0,117.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,208.0,258.0,180.0,49.0,0.0,212.0,0.0,0.0,103.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,365.0,303.0,220.0,130.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,181.0,244.0,57.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
496,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
497,2227.0,2237.0,2075.0,1792.0,1446.0,1276.0,1203.0,1160.0,935.0,614.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
498,1022.0,829.0,705.0,640.0,446.0,173.0,112.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Shortage


,0,1,2,3,4,5,6,7,8,9,...,16,17,18,19,20,21,22,23,24,25
0,0.0,0.0,0.00,0.000,0.0000,38.00000,103.000000,3.000000,113.000000,249.000000,...,85.000000,47.000000,154.000000,80.000000,13.000000,255.000000,32.000000,120.000000,233.000000,130.000000
1,0.0,0.0,0.00,0.000,0.0000,45.00000,153.500000,164.450000,121.115000,325.780500,...,298.957445,268.270212,335.789148,404.052404,353.836683,457.685678,328.379974,299.865982,371.906187,422.334331
2,0.0,0.0,0.00,0.000,82.0000,208.20000,95.820000,262.582000,26.258200,2.625820,...,198.590245,164.859025,56.485902,322.648590,82.264859,104.226486,263.422649,201.342265,80.134226,311.013423
3,0.0,0.0,0.00,0.000,0.0000,0.00000,0.000000,0.000000,0.000000,-0.000000,...,300.000000,352.000000,695.000000,792.000000,912.000000,923.000000,1020.000000,1122.000000,1386.000000,1469.000000
4,0.0,0.0,0.00,0.000,0.0000,0.00000,0.000000,0.000000,0.000000,-0.000000,...,499.000000,600.000000,640.000000,717.000000,717.000000,857.000000,1047.000000,1083.000000,1083.000000,1220.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,0.0,0.0,0.00,0.000,0.0000,0.00000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
496,4.0,0.0,0.00,0.000,0.0000,0.00000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,97.000000,57.500000,328.750000
497,0.0,0.0,0.00,0.000,0.0000,0.00000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,135.000000
498,0.0,0.0,0.00,0.000,0.0000,0.00000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [14]:
print(f'TotalPurchaseCost    :{TotalPurchaseCost.getValue():.2f}')
print(f'TotalShipFixedCost   :{TotalShipFixedCost.getValue():.2f}')
print(f'TotalShipVarCost     :{TotalShipVarCost.getValue():.2f}')
print(f'TotalHoldingCost     :{TotalHoldingCost.getValue():.2f}')
print(f'TotalContainerCost   :{TotalContainerCost.getValue():.2f}')
print(f'TotalBackOrderCost   :{TotalBackOrderCost.getValue():.2f}')
print(f'TotalLostSaleCost    :{TotalLostSaleCost.getValue():.2f}')
print(f'TotalVendorFixedCost :{TotalVendorFixedCost.getValue():.2f}')
print(f'objective-value      :{model.ObjVal:.2f}')

TotalPurchaseCost    :421446110.00
TotalShipFixedCost   :4470.00
TotalShipVarCost     :3200862.00
TotalHoldingCost     :28340011.50
TotalContainerCost   :19032750.00
TotalBackOrderCost   :10753630.85
TotalLostSaleCost    :61799367.77
TotalVendorFixedCost :143600.00
objective-value      :544720802.12
